In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
import urllib
import json
import datetime
import pandas as pd
from pandas.io.json import json_normalize
import math
import time
from math import cos
from pandas import DataFrame

In [2]:
map_df = pd.read_csv('TorontoMap.csv')

In [3]:
map_df.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [6]:
#Foursquare Credentials
CLIENT_ID = 'PTX5DHM4UGPITUVP1GY45OW0QT3QEKNKTWK00BD44UIW1UES' # Foursquare ID
CLIENT_SECRET = 'WXXXJLFVJY5PNX2C10M5XX3MOSV30JXLKNX1BBB23RN3UL5V' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('client_id: ' + CLIENT_ID)
print('client_secret:' + CLIENT_SECRET)

Your credentails:
client_id: PTX5DHM4UGPITUVP1GY45OW0QT3QEKNKTWK00BD44UIW1UES
client_secret:WXXXJLFVJY5PNX2C10M5XX3MOSV30JXLKNX1BBB23RN3UL5V


In [51]:
LIMIT = 100
radius = 500
lat = 43.654260
lng = -79.360636
category_id = '4bf58dd8d48988d1fa941735'

# Repeat the function to all the neighbourhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            category_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue ID',
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
Toronto_venues = getNearbyVenues(names=map_df['Neighbourhood'],
                                   latitudes=map_df['Latitude'],
                                   longitudes=map_df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront


KeyError: 'stats'

In [9]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Harbourfront,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
1,Regent Park,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
2,Ryerson,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market
3,Ryerson,43.657162,-79.378937,Fruit Express Plus,43.659182,-79.382910,4c1e1bf663750f476e30b967,Farmers Market
4,Garden District\n,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market


In [10]:
fm_df = Toronto_venues[Toronto_venues['Venue Category'].str.contains("Farmers Market")].reset_index(drop=True)

fm_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,Harbourfront,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
1,Regent Park,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
2,Ryerson,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market
3,Ryerson,43.657162,-79.378937,Fruit Express Plus,43.659182,-79.382910,4c1e1bf663750f476e30b967,Farmers Market
4,Garden District\n,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market
5,Garden District\n,43.657162,-79.378937,Fruit Express Plus,43.659182,-79.382910,4c1e1bf663750f476e30b967,Farmers Market
6,St. James Town,43.651494,-79.375418,St. Lawrence Market (North Building),43.648793,-79.371945,4bbe53a1848da593e522ea6d,Farmers Market
7,St. James Town,43.651494,-79.375418,St. Lawrence Market (South Building),43.648743,-79.371597,4ad4c062f964a520fbf720e3,Farmers Market
8,St. James Town,43.651494,-79.375418,Market Square,43.649231,-79.373001,4e42dd7eb0fbe97644d1b1b7,Farmers Market
9,St. James Town,43.651494,-79.375418,Ponesse Foods,43.648964,-79.371615,4beeb10635dc0f47261b405b,Farmers Market


In [54]:
fm_df.to_csv('FarmersMarketDF.csv')

NameError: name 'fm_df' is not defined

In [3]:
fm_df = pd.read_csv('FarmersMarketDF.csv')
fm_df.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,0,Harbourfront,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
1,1,Regent Park,43.654260,-79.360636,Distillery Sunday Market,43.650075,-79.361832,4c3e1eaa6faac9b66dc60d76,Farmers Market
2,2,Ryerson,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market
3,3,Ryerson,43.657162,-79.378937,Fruit Express Plus,43.659182,-79.382910,4c1e1bf663750f476e30b967,Farmers Market
4,4,Garden District\n,43.657162,-79.378937,The Farmers’ Market at Ryerson,43.657871,-79.379506,4e01172b88776240d3f3f53d,Farmers Market


In [58]:
#def getvenuesstats(venue_id): 
    
id_list = ['4c1e1bf663750f476e30b967', '4e01172b88776240d3f3f53d', '4c3e1eaa6faac9b66dc60d76']

for x in (id_list):
           
        print(x)
        tips='https://api.foursquare.com/v2/venues/'
        params=dict(client_id=CLIENT_ID,client_secret=CLIENT_SECRET,v=VERSION)
        resp_tips=requests.get(url=tips+x+"/likes", params=params).json()['response']['likes']['count']
        id_list.append([{'Venue' : x, 
            'Likes' : resp_tips}])
    
    
s_venues = pd.DataFrame([item for id_list in id_list for item in id_list])
       
return(s_venues)

4c1e1bf663750f476e30b967


KeyError: 'likes'

In [78]:
venues_rating = getvenuesstats(venue_id=fm_df['Venue ID'])

In [83]:
item_list = []

for item in fm_df['Venue ID']:

    s = item
    tips='https://api.foursquare.com/v2/venues/'
    params=dict(client_id=CLIENT_ID,client_secret=CLIENT_SECRET,v=VERSION)
    resp_tips=requests.get(url=tips+s, params=params).json()['response']['venue']['rating']
    print(resp_tips)
    
    if resp_tips == 0:
        resp_tips = 0
        
    else:
        fm_df['Ratings'] = item_list.append([resp_tips])
        #s_venues = likes_list.append([{'Venue ID' : venue_id, 'Likes' : resp_tips}])


return fm_df

9.4
9.4


KeyError: 'rating'

In [68]:

def reviews(venue_id):
    
    s_venues = pd.DataFrame(columns=['Venue ID','Ratings'])
    
    likes_list=[]

    for item in venue_id:

        #s = item
        tips='https://api.foursquare.com/v2/venues/'
        params=dict(client_id=CLIENT_ID,client_secret=CLIENT_SECRET,v=VERSION)
        resp_tips=requests.get(url=tips+s, params=params).json()['response']['venue']['rating']
        print(resp_tips)
    
    s_venues = likes_list.append([{'Venue ID' : venue_id, 'Ratings' : resp_tips}])


return s_venues


NameError: name 's_venues' is not defined

In [165]:
new_df = reviews(venue_id=fm_df['Venue ID'])

KeyError: 'likes'

In [159]:
new_df

NameError: name 'new_df' is not defined

In [128]:
s = '4bbe53a1848da593e522ea6d'
urlstr='https://api.foursquare.com/v2/venues/'
params=dict(client_id=CLIENT_ID,client_secret=CLIENT_SECRET,v=VERSION, limit=50)
resp_tips=requests.get(url=urlstr+s+'/tips', params=params).json()['response']
print(resp_tips)

{}


In [ ]:
for item in fs_checkin_data['response']['checkins']['items']:
    venue = item['venue']
    print venue['canonicalUrl'], venue['name']

SyntaxError: invalid syntax (<ipython-input-31-9e9a6b2829ec>, line 4)

In [ ]:
df_iceland.plot(kind='bar', figsize=(10, 6))

plt.xlabel('Year') # add to x-label to the plot
plt.ylabel('Number of immigrants') # add y-label to the plot
plt.title('Icelandic immigrants to Canada from 1980 to 2013') # add title to the plot

plt.show()